In [1]:
!pip install fasttext

You should consider upgrading via the '/Users/przivic/anaconda3/bin/python -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
df = pd.read_csv('/Users/przivic/Downloads/kaggle_dataset/AllMoviesDetailsCleaned.csv', delimiter=';')

/Users/przivic/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
from dateutil.parser import parse

df = df[~df.release_date.isna()].copy()
df['release_year'] = df.release_date.apply(lambda x: parse(x).year)
df = df[df.release_year > 1970].copy()
df = df[df.vote_average > 0].copy()
df = df[~df.overview.isna()].copy()

len(df)

93787

In [4]:
train = df[df.release_year <= 2015]
test = df[df.release_year > 2015]

len(test) / len(train), len(test)

(0.06719237158917639, 5905)

In [5]:
X_train = train.overview.tolist()
X_test = test.overview.tolist()

y_train = train.vote_average > 7.5
y_test = test.vote_average > 7.5

In [9]:
!head cooking.stackexchange.txt

__label__sauce __label__cheese How much does potato starch affect a cheese sauce recipe?
__label__food-safety __label__acidity Dangerous pathogens capable of growing in acidic environments
__label__cast-iron __label__stove How do I cover up the white spots on my cast iron stove?
__label__restaurant Michelin Three Star Restaurant; but if the chef is not there
__label__knife-skills __label__dicing Without knife skills, how can I quickly and accurately dice vegetables?
__label__storage-method __label__equipment __label__bread What's the purpose of a bread box?
__label__baking __label__food-safety __label__substitutions __label__peanuts how to seperate peanut oil from roasted peanuts at home?
__label__chocolate American equivalent for British chocolate terms
__label__baking __label__oven __label__convection Fan bake vs bake
__label__sauce __label__storage-lifetime __label__acidity __label__mayonnaise Regulation and balancing of readymade packed mayonnaise and other sauces


In [10]:
def create_file(x, y, fname):
    with open(fname, 'w') as f:
        for i, x_i in enumerate(x):
            if i > 0: f.write('\n')
            f.write(f'__label__{y[i]} {x_i}')

In [52]:
def predict(model, X):
    probs = []
    for labels, scores in zip(*model.predict(X, k=2)):
        probs.append(dict(zip(labels, scores))['__label__True'])
    return probs

In [12]:
create_file(X_train, y_train.tolist(), 'train.data')
create_file(X_test, y_test.tolist(), 'test.data')

In [60]:
import fasttext

model = fasttext.train_supervised(input='train.data')

In [61]:
from sklearn.metrics import roc_auc_score
import numpy as np

print('train', roc_auc_score(y_train, predict(model, X_train)))
print('test', roc_auc_score(y_test, predict(model, X_test)))

train 0.8080887512380266
test 0.6214995432672069


In [55]:
model = fasttext.train_supervised(input='train.data', minn=3, maxn=6)

In [57]:
from sklearn.metrics import roc_auc_score
import numpy as np

print('train', roc_auc_score(y_train, predict(model, X_train)))
print('test', roc_auc_score(y_test, predict(model, X_test)))

train 0.7137659141943012
test 0.648415529432183


In [58]:
model = fasttext.train_supervised(input='train.data', minn=3, maxn=6, wordNgrams=3)

In [59]:
from sklearn.metrics import roc_auc_score
import numpy as np

print('train', roc_auc_score(y_train, predict(model, X_train)))
print('test', roc_auc_score(y_test, predict(model, X_test)))

train 0.7154571106107204
test 0.6491392763479077


In [62]:
model = fasttext.train_supervised(input='train.data', minn=3, maxn=6, epoch=10)

In [63]:
from sklearn.metrics import roc_auc_score
import numpy as np

print('train', roc_auc_score(y_train, predict(model, X_train)))
print('test', roc_auc_score(y_test, predict(model, X_test)))

train 0.7815941936127887
test 0.6380702946106105


In [64]:
model = fasttext.train_supervised(input='train.data', minn=3, maxn=6, epoch=3)

In [65]:
from sklearn.metrics import roc_auc_score
import numpy as np

print('train', roc_auc_score(y_train, predict(model, X_train)))
print('test', roc_auc_score(y_test, predict(model, X_test)))

train 0.6800508476435259
test 0.6421109215600578
